In [1]:
import numpy as np
from libsvm.svmutil import svm_problem, svm_parameter, svm_train, svm_save_model
from discrete_wavelet_transform import dwt_2d
from local_binary_pattern import lbp
from feature_fusion import concatenate_lbp_dwt
import time
import os
import matplotlib.pyplot as plt
import cv2

In [ ]:
def get_data(directory):
    # load preprocessed images
    preprocessed_img = []

    for filename in os.listdir(directory):
        image = os.path.join(directory, filename)
        if image is not None:
            img = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
            preprocessed_img.append(img)

    # feature extraction
    print("Performing Feature Extraction")
    time.sleep(1)

    # applying local binary pattern
    print("Applying Local Binary Pattern")
    lbp_img_features = []
    for i in preprocessed_img:
        texture_features = lbp(i)
        print('\n\n')
        print(texture_features)


        # store the features in a lbp_img_features list
        lbp_img_features.append(texture_features)
        print(f"\n{len(lbp_img_features)} out of {len(preprocessed_img)} images\nPercentage: {(float(len(lbp_img_features)) / float(len(preprocessed_img)) * 100)}\n")
    print('\nLBP application finished\n\n')
    
    # applying discrete wavelet transform
    print("Applying DWT to Images")

    dwt_img_features = []
    for i in preprocessed_img:
        freq_features = dwt_2d(i)
        print('\n\n')
        print(freq_features)

        # store the features in a dwt_img_features list
        dwt_img_features.append(cv2.resize(freq_features, dsize=(512, 512)))
        print(f"\n{len(dwt_img_features)} out of {len(preprocessed_img)} images\nPercentage: {(float(len(dwt_img_features)) / float(len(preprocessed_img)) * 100)}\n")
    print("\nDWT application finished\n\n")


    # applying feature fusion
    fused_features = []

    for dwt_features, lbp_features in zip(dwt_img_features, lbp_img_features):
        feature_vector = concatenate_lbp_dwt(lbp_features, dwt_features)
        fused_features.append(feature_vector)
        print(f"\n{len(fused_features)} out of {len(preprocessed_img)} images\nPercentage: {(float(len(fused_features)) / float(len(preprocessed_img)) * 100)}\n")


    return fused_features

In [ ]:
def prepare_data(real, gan):
    print("----------------------------Preparing the Data-------------------------------\n") 
    #label real  and gan datasets
    real_label = np.ones((len(real), 1))
    gan_label = np.zeros((len(gan), 1))


    # combine the labels and datasets
    dataset_labels = np.vstack((real_label, gan_label))
    datasets = np.vstack((real, gan))

    # reshape the labels and datasets for svm requirements
    datasets_final = []

    for i in datasets:
        flattened_feature = i.flatten()
        datasets_final.append(flattened_feature)
    label_final = dataset_labels.reshape(dataset_labels.shape[0])

    print("Labels: ", len(label_final))
    print("Datasets: ", len(datasets_final))

    return label_final, datasets_final

In [ ]:
def train_model(label, datasets):
    print("----------------------Model Training--------------------------\n")
    # SVM parameter
    kernel_type = 2
    C = 1.0

    # check if length of datasets is equal to the length of labels
    if len(label) == len(datasets):
        prob = svm_problem(label, datasets)
        validate = svm_parameter(f'-t {kernel_type} -c {C} -v 5')
        param = svm_parameter(f'-t {kernel_type} -c {C}')
        initial_accurary = svm_train(prob, validate)

        model = svm_train(prob, param)
    
    else:
        print("Length of datasets and labels do not match\n")  
        print("Length of Datasets: ", len(datasets))
        print("Length of Labels: ", len(label))

    return model

In [ ]:
def visualize(real, gan):

    mean1 = [np.mean(features) for features in real]
    mean2 = [np.mean(features) for features in gan]

    plt.plot(mean1, label="real", color="blue")
    plt.plot(mean2, label="gan", color="red")

    # Adding labels and a title
    plt.xlabel('Index')
    plt.ylabel('Mean Value')
    plt.title('Mean Values for Two Classes')


    # Display the plot
    plt.show()

In [ ]:
# provide directory for preprocessed real and gan images
real_directory = "/Users/Danniel/Downloads/preprocessed_real"
gan_directory = "/Users/Danniel/Downloads/preprocessed_gan"

In [ ]:
# run data preparation
real_data = get_data(real_directory)
gan_data = get_data(gan_directory)

In [ ]:
# visualize
visualize(real_data, gan_data)

In [ ]:
# prepare the data
labels, datasets = prepare_data(real_data, gan_data)

In [ ]:
# train the data
model = train_model(labels, datasets )

In [ ]:
# save the model
model_file = "/Users/Danniel/Downloads/faces_validate.model"
svm_save_model(model_file, model)

In [ ]:
from preprocessing import preprocessing
from libsvm.svmutil import svm_load_model, svm_predict

# test the model
def predict(directory):
    images = []

    # load the images and store in images list
    for filename in os.listdir(directory):
        image = os.path.join(directory, filename)
        if image is not None:
            images.append(image)

    print(len(images))        


    # load the model
    model_file = "/Users/Danniel/Downloads/faces.model"
    loaded_model = svm_load_model(model_file)

    # preprocessing
    preprocessed_img = []
    for i in images:
        preprocessed_img.append(preprocessing(i))      


    #  discrete wavelet transform
    dwt_feature = []
    print("\n\n-------------------DWT----------------------------\n")
    for i in preprocessed_img:
        dwt_feature.append(dwt_2d(i))
        print(f"\n{len(dwt_feature)} out of {len(preprocessed_img)} images\nPercentage: {(float(len(dwt_feature)) / float(len(preprocessed_img)) * 100)}\n")


    # local binary pattern
    lbp_feature = []
    print("\n\n-------------------LBP----------------------------\n")
    for i in preprocessed_img:
        print("LBP Features:\n", lbp(i))
        print("\n")
        lbp_feature.append(lbp(i))
        print(f"\n{len(lbp_feature)} out of {len(preprocessed_img)} images\nPercentage: {(float(len(lbp_feature)) / float(len(preprocessed_img)) * 100)}\n")


    # feature fusion
    fused_vector = []
    print("\n\n-------------------FEATURE FUSION----------------------------\n")
    for frequency, texture in zip(dwt_feature, lbp_feature):
        print("Fused Features:\n", concatenate_lbp_dwt(texture, frequency))
        print("\n")
        fused_vector.append(concatenate_lbp_dwt(texture, frequency))
        print(f"\n{len(fused_vector)} out of {len(images)} images\nPercentage: {(float(len(fused_vector)) / float(len(preprocessed_img)) * 100)}\n")

    # flatten the feature vector
    feature_vector = []
    for i in fused_vector:
        feature_vector.append(i.flatten())


    # predict the result
    print("\n\n-------------------THE MODEL IS PREDICTING----------------------------\n")
    predicted_labels, _, _ = svm_predict([], feature_vector, loaded_model, '-q')


    print("------------------------------------------RESULT-----------------------------------\n")
    result = []
    for i in predicted_labels:
        if i == 1.0:
            result.append("Real")
        elif i == 0.0:
            result.append("GAN")

    print(result)

#provide directory for testing dataset
dir = ""

predict(dir)
